# Pretrain attractor network PCP



## Master config

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE", '\n')
!ls

from misc import model_cfg

cfg = model_cfg(code_name='p_task_v0.03', 
                sample_name='hs04', 
                sample_rng_seed=500,
                hidden_units=150, 
                pho_units=250, 
                cleanup_units=50,
                embed_attractor=None,
                w_oh_noise=0.,   # Noise at weight not implement yet
                w_hp_noise=0.,
                w_pp_noise=0.,
                w_pc_noise=0.,
                w_cp_noise=0.,
                act_p_noise=0., # Noise at activation is OK
                tau=0.2, 
                unit_time=4., 
                n_mil_sample=1., 
                batch_size=64, 
                rnn_activation = 'sigmoid',
                w_initializer = 'glorot_uniform',
                learning_rate=0.005,
                save_freq=5)


# Modeling

## Build model
Fully custom 4-layer recurrent time-averaged input

In [ ]:
def buildModel(training=True):
    from tensorflow.keras import Model
    from tensorflow.keras.layers import Layer, Input
    from custom_layer import rnn_pho_task
    from tensorflow.keras.optimizers import Adam

    # Train/test mode checking
    if training is True:
        cfg.w_oh_noise = cfg.w_oh_noise_backup
        cfg.w_hp_noise = cfg.w_hp_noise_backup
        cfg.w_pp_noise = cfg.w_pp_noise_backup
        cfg.w_pc_noise = cfg.w_pc_noise_backup
        cfg.w_cp_noise = cfg.w_cp_noise_backup
        cfg.act_p_noise = cfg.act_p_noise_backup
    else:
        cfg.w_oh_noise = 0
        cfg.w_hp_noise = 0
        cfg.w_pp_noise = 0
        cfg.w_pc_noise = 0
        cfg.w_cp_noise = 0
        cfg.act_p_noise = 0

    input_o = Input(shape=(cfg.pho_units,))
    rnn_model = rnn_pho_task(cfg)(input_o)
    model = Model(input_o, rnn_model)

    adam = Adam(learning_rate=cfg.learning_rate, beta_1=0.9, beta_2=0.999, amsgrad=False)

    model.compile(loss='binary_crossentropy',
                optimizer=adam,
                metrics=['BinaryAccuracy', 'mse'])

    model.summary()

    return model

model = buildModel(training=True)


## Train model

In [ ]:
from keras.callbacks import ModelCheckpoint
from data_wrangling import sampleGenerator
import h5py, pickle, os
from IPython.display import clear_output

from data_wrangling import my_data
data = my_data(cfg)

checkpoint = ModelCheckpoint(cfg.path_weights_checkpoint, verbose=1, 
                             period=cfg.save_freq, save_weights_only=True) 

history = model.fit(sampleGenerator(data.y_train, data.y_train, 
                                    cfg.n_timesteps - 14, cfg.batch_size, 
                                    data.sample_p, cfg.sample_rng_seed),
                                    steps_per_epoch = cfg.steps_per_epoch,
                                    epochs = cfg.nEpo,
                                    verbose=0,  
                                    callbacks=[checkpoint])


# Saving history and model
pickle_out = open(cfg.path_history_pickle,"wb")
pickle.dump(history.history, pickle_out)
pickle_out.close()

clear_output()
print('Training done')

# Reporting

Training history

In [ ]:
from my_eval import training_history

hist = training_history(cfg.path_history_pickle)
hist.plot_all(cfg.path_plot_folder + 'history.html')

Weight matrixs

In [ ]:
from misc import plot_variables
plot_variables(model)

Prediction

In [ ]:
from my_eval import get_all_pronunciations_fast
y_pred_matrix = model.predict(data.y_strain)
y_pred = get_all_pronunciations_fast(y_pred_matrix[-1], data.phon_key)
#y_pred = get_all_pronunciations_fast(y_pred_matrix, data.phon_key)
y_true = get_all_pronunciations_fast(data.y_strain, data.phon_key)

print('\n Y-predicts are: \n')
print(y_pred)
print('\n Y-trues are: \n')
print(y_true)

In [ ]:
!jupyter nbconvert --to html --output-dir=$cfg.path_model_folder attractor.ipynb